# Scaling out numerical computing in Julia
Przemysław Szufel




<a class="anchor" id="toc"></a>
## Table of content

    
1. [Parallelize via Single Instruction Multiple Data (SIMD)](#simd)
2. [Green threading](#green)
3. [Multithreading](#multithreading)
4. [Multi-processing and distributed computing](#multiprocessing)

Before running Jupyter notebook set in Julia number of threads.
This should be done *before* actually running the `notebook()` command.
The number of threads can be also set up in Julia options in Visual Studio code (if this is used to run this notebook).
```
# run this code from Julia console just before starting Jupyter Notebook
ENV["JULIA_NUM_THREADS"]=4
```

In [26]:
println("Number of threads that your Julia is run: ## $(Threads.nthreads())")

Number of threads that your Julia is run: ## 4


In [27]:
using BenchmarkTools, Distributed

<a class="anchor" id="simd"></a>
### Parallelize via Single Instruction Multiple Data (SIMD)
---- [Return to table of contents](#toc) ---



In [28]:
function dot1(x, y)
    s = 0.0
    for i in 1:length(x)
        @inbounds s += x[i]*y[i]
    end
    s
end

dot1 (generic function with 1 method)

In [29]:
function dot2(x, y)
    s = 0.0
    @simd for i in 1:length(x)
        @inbounds s += x[i]*y[i]
    end
    s
end

dot2 (generic function with 1 method)

In [30]:
x = 100*rand(10000)
y = 100*rand(10000);

@btime dot1($x, $y)
@btime dot2($x, $y)

  4.000 μs (0 allocations: 0 bytes)
  763.063 ns (0 allocations: 0 bytes)


2.4943127090471864e7

In [31]:
res1 =  dot1(x, y)

2.4943127090471815e7

In [32]:
res2 =  dot2(x, y)

2.4943127090471864e7

In [33]:
res1 == res2

false

In [34]:
@show res1 
@show res2

res1 = 2.4943127090471815e7
res2 = 2.4943127090471864e7


2.4943127090471864e7

<a class="anchor" id="green"></a>
### Green threading 
---- [Return to table of contents](#toc) ---


In [35]:
@time sleep(2)

  2.015877 seconds (43 allocations: 1024 bytes)


In [36]:
@time t = @async sleep(4)

  0.013347 seconds (2.80 k allocations: 201.922 KiB, 99.11% compilation time)


Task (runnable) @0x0000029eabec9940

In [37]:
t

Task (runnable) @0x0000029eabec9940

In [38]:
function dojob(i)
    val = round(rand(), digits=2)
    sleep(val)   # this could be external computations with I/O
    i, val
end

dojob (generic function with 1 method)

In [39]:
result = Vector{Tuple{Int,Float64}}(undef, 8)

8-element Vector{Tuple{Int64, Float64}}:
 (1, 1.0e-323)
 (3, 2.0e-323)
 (5, 3.0e-323)
 (7, 4.0e-323)
 (9, 5.0e-323)
 (11, 8.4e-323)
 (18, 9.4e-323)
 (20, 1.04e-322)

In [40]:
@time for i=1:8
    result[i] = dojob(i)
end
result

  1.985352 seconds (622 allocations: 40.812 KiB, 0.52% compilation time)


8-element Vector{Tuple{Int64, Float64}}:
 (1, 0.59)
 (2, 0.32)
 (3, 0.06)
 (4, 0.03)
 (5, 0.18)
 (6, 0.03)
 (7, 0.0)
 (8, 0.73)

In [41]:
result = Vector{Tuple{Int,Float64}}(undef, 8);
@time for i=1:8
   @async result[i] = dojob(i)
end
result

  0.000098 seconds (81 allocations: 7.055 KiB)


8-element Vector{Tuple{Int64, Float64}}:
 (140704098874896, 6.95170614788753e-310)
 (140704098866272, 6.95170614788437e-310)
 (140704098874896, 1.424167516463e-311)
 (2882547144272, 1.4241675165105e-311)
 (2882547144400, 1.4241675165737e-311)
 (140704098866272, 1.424167249557e-311)
 (140704098874896, 6.95170614788753e-310)
 (2882538132032, 6.9517061483136e-310)

In [42]:
result


8-element Vector{Tuple{Int64, Float64}}:
 (140704098874896, 6.95170614788753e-310)
 (140704098866272, 6.95170614788437e-310)
 (140704098874896, 1.424167516463e-311)
 (2882547144272, 1.4241675165105e-311)
 (2882547144400, 1.4241675165737e-311)
 (140704098866272, 1.424167249557e-311)
 (140704098874896, 6.95170614788753e-310)
 (2882538132032, 6.9517061483136e-310)

In [43]:
result = Vector{Tuple{Int,Float64}}(undef, 8);
@time @sync for i=1:8
   @async result[i] = dojob(i)
end
result

  0.822314 seconds (1.51 k allocations: 82.570 KiB, 3.52% compilation time)


8-element Vector{Tuple{Int64, Float64}}:
 (1, 0.14)
 (2, 0.08)
 (3, 0.8)
 (4, 0.82)
 (5, 0.56)
 (6, 0.72)
 (7, 0.79)
 (8, 0.63)

#### Programming a simple web server
You should be able to connect using the address <a href="http://localhost:9992/3+4" target="about:blank">http://localhost:9992/3+4</a>

To stop web server click <a href="http://localhost:9992/stopme" target="about:blank">http://localhost:9992/stopme</a>

In [44]:
using Sockets
println("Starting the web server...")
server = Sockets.listen(9992)

Starting the web server...


LoadError: IOError: listen: address already in use (EADDRINUSE)

In [45]:
@async begin
    contt = Ref(true)
    while contt[]
        sock = Sockets.accept(server)
        @async begin
            data = readline(sock)
            print("Got request:\n", data, "\n")
            cmd = split(data, " ")[2][2:end]
            println(sock, "\nHTTP/1.1 200 OK\nContent-Type: text/html\n")
            contt[] = contt[] && (!occursin("stopme", data))
            if contt[]
                 println(sock, string("<html><body>", cmd, "=", 
                     eval(Meta.parse(cmd)), "</body></html>"))
            else
                println(sock,"<html><body>stopping</body></html>")
            end
            close(sock)
        end
    end
    println("Handling requests stopped")
end

Task (failed) @0x0000029f1b8bb840
UndefVarError: `server` not defined
Stacktrace:
 [1] (::var"#7#9")()
   @ Main .\In[45]:4

<a class="anchor" id="multithreading"></a>
### Multithreading
---- [Return to table of contents](#toc) ---


In [46]:
Threads.nthreads()

4

In [47]:
function ssum(x)
    r, c = size(x)
    y = zeros(c)
    for i in 1:c
        for j in 1:r
            @inbounds y[i] += x[j, i]
        end
    end
    y
end

ssum (generic function with 1 method)

In [48]:
function tsum(x)
    r, c = size(x)
    y = zeros(c)
    Threads.@threads for i in 1:c
        for j in 1:r
            @inbounds y[i] += x[j, i]
        end
    end
    y
end


tsum (generic function with 1 method)

In [49]:
x = rand(1000,10000);

In [50]:
@time ssum(x)
@time ssum(x);

  0.029085 seconds (6.63 k allocations: 533.969 KiB, 56.86% compilation time)
  0.011706 seconds (2 allocations: 78.172 KiB)


In [51]:
@time tsum(x)
@time tsum(x);

  0.109102 seconds (44.38 k allocations: 3.117 MiB, 132.62% compilation time)
  0.005122 seconds (30 allocations: 81.047 KiB)


#### Locking mechanism for threads

In [52]:
function f_bad()
    x = 0
    Threads.@threads for i in 1:10^6
        x += 1
    end
    return x
end


f_bad (generic function with 1 method)

In [53]:
f_bad()

250346

In [54]:
function f_add()
    x = 0 
    for i in 1:10^6
        x += 1
    end
    x
end
@btime f_add()
    

  1.400 ns (0 allocations: 0 bytes)


1000000

In [55]:
function f_atomic()
    x = Threads.Atomic{Int}(0)
    Threads.@threads for i in 1:10^6
        Threads.atomic_add!(x, 1)
    end
    return x[]
end
f_atomic()

1000000

In [56]:
function f_spin()
    l = Threads.SpinLock()
    x = 0
    Threads.@threads for i in 1:10^6
        Threads.lock(l) do
            x += 1
        end
    end
    return x
end

function f_reentrant()
    l = ReentrantLock()
    x = 0
    Threads.@threads for i in 1:10^6
        Threads.lock(l) do
            x += 1
        end
    end
    return x
end


f_reentrant (generic function with 1 method)

In [57]:
using DataFrames
stats = DataFrame()
for f in [f_bad, f_atomic, f_spin, f_reentrant]
    for i = 1:2
        value, elapsedtime  = @timed f()
        push!(stats, (f=string(f),i=i, value=value, timems=elapsedtime*1000))
    end
end
println(stats)


8×4 DataFrame
 Row │ f            i      value    timems   
     │ String       Int64  Int64    Float64  
─────┼───────────────────────────────────────
   1 │ f_bad            1   251023   35.483
   2 │ f_bad            2   252967   35.2314
   3 │ f_atomic         1  1000000   20.4998
   4 │ f_atomic         2  1000000   18.539
   5 │ f_spin           1  1000000  465.34
   6 │ f_spin           2  1000000  257.439
   7 │ f_reentrant      1  1000000  480.495
   8 │ f_reentrant      2  1000000  385.128


<a class="anchor" id="multiprocessing"></a>
### Multi-processing and distributed computing
---- [Return to table of contents](#toc) ---


In [58]:
using Distributed

This code adds 4 workers (and avoids adding more)

In [59]:
addprocs(max(0, 5-nprocs()));

In [60]:
workers()

4-element Vector{Int64}:
 2
 3
 4
 5

In [61]:
function s_rand()
    n = 10^4
    x = 0.0
    for i in 1:n
        x += sum(rand(10^4))
    end
    x / n
end
 
@time s_rand()
@time s_rand()


  0.920285 seconds (20.00 k allocations: 763.397 MiB, 25.61% gc time)
  0.725482 seconds (20.00 k allocations: 763.397 MiB, 17.42% gc time)


5000.044150780158

In [62]:
using Distributed
 
function p_rand()
    n = 10^4
    x = @distributed (+) for i in 1:n
        # the last line will be aggregated
        sum(rand(10^4))
    end
    x / n
end

@time p_rand()
@time p_rand()


  1.612347 seconds (463.07 k allocations: 31.114 MiB, 47.33% compilation time)
  0.173494 seconds (426 allocations: 24.016 KiB)


4999.3877325494

In [63]:
workers()'

1×4 adjoint(::Vector{Int64}) with eltype Int64:
 2  3  4  5

In [64]:
fetch(@spawnat 3 4+3)

7

In [65]:
function myf() 
    println("I am on worker ", myid())
    rand()
end
myf()

I am on worker 1


0.21248315596630618

In [66]:
a = nothing
try 
    fetch(@spawnat 4 myf())
catch e
    println(e)
end

RemoteException(4, CapturedException(UndefVarError(Symbol("#myf")), Any[(deserialize_datatype at Serialization.jl:1399, 1), (handle_deserialize at Serialization.jl:867, 1), (deserialize at Serialization.jl:814, 1), (handle_deserialize at Serialization.jl:874, 1), (deserialize at Serialization.jl:814 [inlined], 1), (deserialize_global_from_main at clusterserialize.jl:160, 1), (#5 at clusterserialize.jl:72 [inlined], 1), (foreach at abstractarray.jl:3086, 1), (deserialize at clusterserialize.jl:72, 1), (handle_deserialize at Serialization.jl:960, 1), (deserialize at Serialization.jl:814, 1), (handle_deserialize at Serialization.jl:871, 1), (deserialize at Serialization.jl:814, 1), (handle_deserialize at Serialization.jl:874, 1), (deserialize at Serialization.jl:814 [inlined], 1), (deserialize_msg at messages.jl:87, 1), (#invokelatest#2 at essentials.jl:887 [inlined], 1), (invokelatest at essentials.jl:884 [inlined], 1), (message_handler_loop at process_messages.jl:176, 1), (process_tcp_s

In [67]:
@everywhere function myf() 
    println("I am on worker ", myid())
    rand()
end
fetch(@spawnat 4 myf())

      From worker 4:	I am on worker 4


0.7642915859931477

#### A typical pattern for setting an intial state across workers

In [68]:
using Distributed
@everywhere using Pkg
@everywhere Pkg.activate(".")
@everywhere using Distributed, Random, DataFrames

@everywhere function calc(x, y)
    2x + y
end

@everywhere function init_worker()    
   Random.seed!(myid())
    # reading initial data from files or other actions
end

@sync for wid in workers()
    @async fetch(@spawnat wid init_worker())
end


      From worker 5:	  Activating project at `C:\AAABIBLIOTEKA\Berkeley`
      From worker 2:	  Activating project at `C:\AAABIBLIOTEKA\Berkeley`
      From worker 3:	  Activating project at `C:\AAABIBLIOTEKA\Berkeley`
      From worker 4:	  Activating project at `C:\AAABIBLIOTEKA\Berkeley`


  Activating project at `C:\AAABIBLIOTEKA\Berkeley`


Typically results are collected to a `DataFrame`

In [69]:
data = @distributed (append!) for (i, j) = vec(collect(Iterators.product(1:4, 1:3)))
    a = rand(1:499)
    b = rand(1:9)*1000
    c = calc(a, b)
    DataFrame(;i,j,a,b,c,procid = myid())
end

Row,i,j,a,b,c,procid
,Int64,Int64,Int64,Int64,Int64,Int64
1,1,1,217,1000,1434,2
2,2,1,125,6000,6250,2
3,3,1,39,5000,5078,2
4,4,1,363,3000,3726,3
5,1,2,370,6000,6740,3
6,2,2,32,2000,2064,3
7,3,2,76,4000,4152,4
8,4,2,82,8000,8164,4
9,1,3,441,4000,4882,4
